### Data Loading 

In [4]:
pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import numpy as np
import pandas as pd

In [6]:
sales = pd.read_csv(r'C:\Users\wwwsu\Desktop\All folders\Logistics_demand\data\raw\sales_train_evaluation.csv')
price = pd.read_csv(r'C:\Users\wwwsu\Desktop\All folders\Logistics_demand\data\raw\sell_prices.csv')
calendar = pd.read_csv(r'C:\Users\wwwsu\Desktop\All folders\Logistics_demand\data\raw\calendar.csv')

### Optimizing Sales dataframe

In [7]:
# Convert identifier columns to category
id_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
for col in id_cols:
    sales[col] = sales[col].astype('category')

# Convert sales columns to smaller int type (e.g., uint16)
sales_cols = sales.columns.difference(id_cols)
sales[sales_cols] = sales[sales_cols].astype(np.uint16)

# Check memory usage before and after
print("Sales Data Memory Usage (MB):", sales.memory_usage(deep=True).sum() / 1024**2)

Sales Data Memory Usage (MB): 116.6247148513794


### Optimizing calendar Dataframe

In [8]:
# Select essential columns only
calendar = calendar[['d', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
                     'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
                     'snap_CA', 'snap_TX', 'snap_WI']]

# Convert d to category
calendar['d'] = calendar['d'].astype('category')

# Convert date to datetime
calendar['date'] = pd.to_datetime(calendar['date'])

# Convert categorical columns
cat_cols = ['weekday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
for col in cat_cols:
    calendar[col] = calendar[col].astype('category')

# Convert numeric columns
calendar['wday'] = calendar['wday'].astype(np.int8)
calendar['month'] = calendar['month'].astype(np.int8)
calendar['year'] = calendar['year'].astype(np.int16)
calendar['snap_CA'] = calendar['snap_CA'].astype(np.int8)
calendar['snap_TX'] = calendar['snap_TX'].astype(np.int8)
calendar['snap_WI'] = calendar['snap_WI'].astype(np.int8)

print("Calendar Data Memory Usage (MB):", calendar.memory_usage(deep=True).sum() / 1024**2)


Calendar Data Memory Usage (MB): 0.22610950469970703


### Optimize Price Dataframe

In [9]:
price['store_id'] = price['store_id'].astype('category')
price['item_id'] = price['item_id'].astype('category')
price['wm_yr_wk'] = price['wm_yr_wk'].astype(np.int16)
price['sell_price'] = price['sell_price'].astype(np.float32)

print("Price Data Memory Usage (MB):", price.memory_usage(deep=True).sum() / 1024**2)


Price Data Memory Usage (MB): 58.96129322052002


### Cleaning the data 

In [10]:
print("Missing values in sales data:")
print(sales.isnull().sum().sum()) 

Missing values in sales data:
0


In [11]:

print("Missing values in price data:")
print(price.isnull().sum())

Missing values in price data:
store_id      0
item_id       0
wm_yr_wk      0
sell_price    0
dtype: int64


In [12]:
print("Missing values in calendar data:")
print(calendar.isnull().sum())

Missing values in calendar data:
d                  0
date               0
wm_yr_wk           0
weekday            0
wday               0
month              0
year               0
event_name_1    1807
event_type_1    1807
event_name_2    1964
event_type_2    1964
snap_CA            0
snap_TX            0
snap_WI            0
dtype: int64


Missing event names/types mean no event on those days, which is expected because most days are normal days without special events. So, these missing values are not errors but represent “no event”.

### Filling missing event values with 'No_event'

In [13]:
calendar['event_name_1'] = calendar['event_name_1'].cat.add_categories('No_event').fillna('No_event')
calendar['event_type_1'] = calendar['event_type_1'].cat.add_categories('No_event').fillna('No_event')

calendar['event_name_2'] = calendar['event_name_2'].cat.add_categories('No_event').fillna('No_event')
calendar['event_type_2'] = calendar['event_type_2'].cat.add_categories('No_event').fillna('No_event')


### Analyze Zero Sales Distribution 
Zero sales days are common and meaningful (e.g., no demand, out-of-stock). Understanding zero sales patterns helps decide if any filtering or special handling is needed.

#### Calculating Percentage of zero sales per item-store

In [14]:
# Melt sales data to long format for easier analysis
sales_long = pd.melt(
    sales,
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='sales'
)

# Calculate zero sales percentage per item-store combination
zero_sales_pct = sales_long.groupby(['item_id', 'store_id'])['sales'].apply(lambda x: (x == 0).mean())

# Summary statistics of zero sales percentage
print(zero_sales_pct.describe())



C:\Users\wwwsu\AppData\Local\Temp\ipykernel_30548\2898919737.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  zero_sales_pct = sales_long.groupby(['item_id', 'store_id'])['sales'].apply(lambda x: (x == 0).mean())


count    30490.000000
mean         0.679978
std          0.223406
min          0.001546
25%          0.534776
50%          0.733127
75%          0.865018
max          0.993818
Name: sales, dtype: float64


Since the median zero sales is 73%, zeros are a significant part of the data and should not be removed blindly.

### Merging Datasets and Aggregating Weekly

#### Merging sales_long with calender

In [15]:
# Merge sales_long with calendar on 'd' to get date and wm_yr_wk
sales_calendar = sales_long.merge(calendar[['d', 'date', 'wm_yr_wk']], on='d', how='left')

print(f"sales_calendar shape: {sales_calendar.shape}")
print(sales_calendar.head())


sales_calendar shape: (59181090, 10)
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id    d  sales       date  wm_yr_wk  
0       CA  d_1      0 2011-01-29     11101  
1       CA  d_1      0 2011-01-29     11101  
2       CA  d_1      0 2011-01-29     11101  
3       CA  d_1      0 2011-01-29     11101  
4       CA  d_1      0 2011-01-29     11101  


#### Merging sales_calendar with price 

In [16]:
# Merge with price on item_id, store_id, and wm_yr_wk
full_data = sales_calendar.merge(price, on=['item_id', 'store_id', 'wm_yr_wk'], how='left')

print(f"full_data shape: {full_data.shape}")
print(full_data.head())


full_data shape: (59181090, 11)
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id    d  sales       date  wm_yr_wk  sell_price  
0       CA  d_1      0 2011-01-29     11101         NaN  
1       CA  d_1      0 2011-01-29     11101         NaN  
2       CA  d_1      0 2011-01-29     11101         NaN  
3       CA  d_1      0 2011-01-29     11101         NaN  
4       CA  d_1      0 2011-01-29     11101         NaN  


#### Aggregating Weekly Sales and Price

In [17]:
weekly_data = full_data.groupby(['item_id', 'store_id', 'wm_yr_wk']).agg({
    'sales': 'sum',
    'sell_price': 'mean'
}).reset_index()

print(f"weekly_data shape: {weekly_data.shape}")
print(weekly_data.head())


C:\Users\wwwsu\AppData\Local\Temp\ipykernel_30548\3845643553.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  weekly_data = full_data.groupby(['item_id', 'store_id', 'wm_yr_wk']).agg({


weekly_data shape: (8476220, 5)
       item_id store_id  wm_yr_wk  sales  sell_price
0  FOODS_1_001     CA_1     11101     10         2.0
1  FOODS_1_001     CA_1     11102      6         2.0
2  FOODS_1_001     CA_1     11103     10         2.0
3  FOODS_1_001     CA_1     11104     13         2.0
4  FOODS_1_001     CA_1     11105     15         2.0


#### Adding calendar Features to weekly data

In [18]:
# Extract unique week info from calendar
week_info = calendar[['wm_yr_wk', 'month', 'year']].drop_duplicates()

# Merge with weekly_data
weekly_data = weekly_data.merge(week_info, on='wm_yr_wk', how='left')


In [20]:
print(weekly_data.head())

       item_id store_id  wm_yr_wk  sales  sell_price  month  year
0  FOODS_1_001     CA_1     11101     10         2.0      1  2011
1  FOODS_1_001     CA_1     11101     10         2.0      2  2011
2  FOODS_1_001     CA_1     11102      6         2.0      2  2011
3  FOODS_1_001     CA_1     11103     10         2.0      2  2011
4  FOODS_1_001     CA_1     11104     13         2.0      2  2011


In [25]:
weekly_data.shape

(10183660, 7)

In [27]:
weekly_data.to_csv('../data/processed/weekly_aggregated_sales.csv', index=False)